# LLaMA-3 Tokenizer
LLaMA는 페이스북에서 2023년 2월에 출시한 공개 LLM 모델입니다. 7B(70억) ~ 65B(650억)에 이르는 다양한 크기의 모델을 허깅페이스에 공개하여 사용할 수 있도록 하였습니다.

공개된 모델이기 때문에 허깅페이스에서 다운로드 받아 사용할 수 있습니다.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# allganize : 국내 LLM 전문 회사입니다. LLaMA-3에 한글 데이터를 부어서 한글로 조금 더 학습이 잘 된 LLM을 제공합니다.
# https://huggingface.co/allganize/Llama-3-Alpha-Ko-8B-Instruct

model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Encode & Decode

In [3]:
print(tokenizer.encode("오늘은 배가 너무 고프다"))

[128000, 58368, 105622, 34804, 74769, 20565, 109748, 46230, 58260, 63644, 13447]


In [4]:
print(
    tokenizer.decode(
        [128000, 58368, 105622, 34804, 74769, 20565, 109748, 46230, 58260, 63644, 13447]
    )
)

<|begin_of_text|>오늘은 배가 너무 고프다


# Template
템플릿이란 모델이 훈련될 때 입력으로 제공되었던 **형식**입니다. 모델마다 모두 다르게 설정되어 있으며, 학습 및 사용 시에 해당 템플릿을 따르지 않으면 제대로 된 성능을 얻기가 힘들다.

## Role 이해하기
최근 개발되는 LLM에는 역할(`role`)이라는 개념이 존재합니다. role은 크게 user, system, assistant가 존재합니다.
- `user` : 사용자 신분
  - 사용자가 챗봇에게 질문할 내용이 됩니다.
- `system` : LLM에게 시키고 싶은 것
  - LLM이 어떤 기준을 가지고 작동할지에 대한 설정이 가능합니다.
- `assistant` : LLM의 답변

개발자가 직접적으로 template을 신경쓰지 않도록 허깅페이스의 모델은 `apply_chat_template`이라는 함수를 통해 모델의 Chat Template을 자동으로 생성할 수 있게 도와줍니다. 때에 따라 이 함수를 사용하지 않고 개발하는 경우도 종종 있습니다.

In [5]:
messages = [
    {
        "role": "system",
        "content": "당신은 인공지능 비서입니다. 묻는 말에 친절하고 정확하게 답변하세요.",
    },
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"},
]

# ChatTemplate 인코딩 & 디코딩

In [6]:
# Tokenizer를 이용해 ChatTemplate 만들기
template_messages = tokenizer.apply_chat_template(messages, tokenize=False)
print(template_messages)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 비서입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|>


In [7]:
# tokenize=True로 설정하면 템플릿 자체가 정수 인코딩 되어 나옵니다.
template_messages = tokenizer.apply_chat_template(messages, tokenize=True)
print(template_messages)

[128000, 128006, 9125, 128007, 271, 65895, 83628, 34804, 59777, 103896, 67119, 75086, 27796, 80052, 13, 100570, 119, 16969, 101264, 19954, 108280, 104834, 101360, 127923, 102893, 111964, 92245, 13, 128009, 128006, 882, 128007, 271, 34804, 101066, 21028, 111902, 104193, 111833, 122115, 114942, 34983, 59269, 246, 128009]


In [8]:
print(tokenizer.decode(template_messages))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 비서입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|>


## `assistant` 적용하기
`assistant` 토큰은 LLM 모델이 여기부터 답변을 하는 구간이다 라는 것을 학습하기 위한 토큰입니다. 따라서 LLM이 학습될 때 다음과 같이 훈련 되었다는 이야기 입니다.
```
<system> 당신은 인공지능 비서입니다 </system>
<user> 손흥민의 소속 팀은? </user>
<assistant> 손흥민의 소속 팀은 토트넘 ~~~ </assistant>
```

따라서 LLM을 이용해 챗봇에 대한 답변을 받기 위해서는 다음과 같이 `assistant`를 추가해서 물어봐야 합니다.
```
<system> 당신은 인공지능 비서입니다 </system>
<user> 손흥민의 소속 팀은? </user>
<assistant>
```

만약 `assistant`를 삽입하지 않고 챗봇에게 요청하면 오동작할 가능성이 생기게 됩니다.
```
<system> 당신은 인공지능 비서입니다 </system>
<user> 손흥민의 소속 팀은? </user>
```

하지만 매번 LLM의 Template에 맞춰서 `assistant`를 넣어주는 것은 매우 번거로운 작업이기 때문에 `add_generation_prompt` 옵션을 이용하여 자동으로 `assistant`를 추가하게 할 수 있습니다.

In [9]:
# add_generation_prompt=False로 적용한 경우
template_messages = tokenizer.apply_chat_template(
    messages, add_generation_prompt=False, tokenize=False
)

print(template_messages)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 비서입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|>


In [10]:
# add_generation_prompt=True 적용한 경우
template_messages = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False
)

print(template_messages)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 비서입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [11]:
# # .env나 보안 파일에서 프롬프트를 불러와서 사용하는 케이스도 있다.
# temp = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

# {system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

# 은행의 기준 금리에 대해서 설명해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>
# """

# LLaMA-3 모델 호출하기

In [13]:
# 비트 양자화를 위한 패키지입니다. 모델의 용량을 줄여주는 역할을 합니다. 용량이 줄어드는 만큼 성능은 줄어들 수 있습니다.
# 설치 후 세션 다시 시작해야합니다.
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.3 MB/s eta 0:00:00


In [12]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

print(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map="auto"
)

allganize/Llama-3-Alpha-Ko-8B-Instruct


config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [13]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Llama

In [14]:
# LLM에는 텍스트 생성이 종료 되었다는 의미의 stop_token이 존재합니다.
# 모델이 글을 쓰다가 멈추는 시점을 결정하는 토큰입니다.
# 모든 모델에 다 있는 것은 아닙니다.
tokenizer.eos_token_id

128001

In [15]:
tokenizer.decode(tokenizer.eos_token_id)

'<|end_of_text|>'

In [16]:
device = "cuda"

messages = [
    {
        "role": "system",
        "content": "당신은 챗봇 한경이 입니다,. 묻는 말에 친절하고 정확하게 답변하세요.",
    },
    {"role": "user", "content": "당신이 누군지 설명해주십시오."},
]

In [17]:
encodes = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"  # 파이토치 텐서로 리턴
)
encodes

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 102783,
            245, 103850,    229,  62398,  66406,  13094, 119519,  17974, 100570,
            119,  16969, 101264,  19954, 108280, 104834, 101360, 127923, 102893,
         111964,  92245,     13, 128009, 128006,    882, 128007,    271,  65895,
          83628,  13094, 106200, 102323,  22035, 114942,  34983,  55430, 119978,
             13, 128009, 128006,  78191, 128007,    271]])

In [18]:
model_inputs = encodes.to(device)
model_inputs

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 102783,
            245, 103850,    229,  62398,  66406,  13094, 119519,  17974, 100570,
            119,  16969, 101264,  19954, 108280, 104834, 101360, 127923, 102893,
         111964,  92245,     13, 128009, 128006,    882, 128007,    271,  65895,
          83628,  13094, 106200, 102323,  22035, 114942,  34983,  55430, 119978,
             13, 128009, 128006,  78191, 128007,    271]], device='cuda:0')

**시스템 프롬프트**
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 챗봇 한경이 입니다. 묻는 말에 친절하고 정확하게 답변하세요<|eot_id|>
```

**유저 프롬프트**
```
<|start_header_id|>user<|end_header_id|>

당신이 누군지 설명해주십시오<|eot_id|>
```

**어시스턴스 시작점**
```
<|start_header_id|>assistant<|end_header_id|>


```

**답변 생성하기**
LLM에서 답변을 받기 위해서는 generate 함수를 사용

In [19]:
generated_ids = model.generate(
    model_inputs,  # 정수 인코딩이 된 프롬프트(Chat Template)
    max_new_tokens=512,  # 모델이 대답할 응답 메시지의 최대 길이
    eos_token_id=tokenizer.eos_token_id,  # 모델의 텍스트 생성 종료 토큰
    repetition_penalty=1.05,  # 반복된 응답 지수. 값이 작아질 수록 반복된 응답을 수행
)

generated_ids

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 102783,
            245, 103850,    229,  62398,  66406,  13094, 119519,  17974, 100570,
            119,  16969, 101264,  19954, 108280, 104834, 101360, 127923, 102893,
         111964,  92245,     13, 128009, 128006,    882, 128007,    271,  65895,
          83628,  13094, 106200, 102323,  22035, 114942,  34983,  55430, 119978,
             13, 128009, 128006,  78191, 128007,    271, 101193, 124409,      0,
         102678,  16969, 102783,    245, 103850,    229,  62398,  66406,  80052,
             13, 102783,    245, 103850,    229,  62398,  66406,  34804,  59777,
         103896,  67119,   4444,     40,      8,  43139, 118667,  86351,  62060,
          57390, 102193, 106562,    229,  43139,     11,  41820,  26799,  81673,
          62060, 117216, 110155,  61139,  18918, 108273, 101360,     11, 109760,
          19954, 108386, 108859,     11, 118696,  56773,  38187,  19954, 112107,
         116844,  48936,  29

In [20]:
# batch_decode : 여러 개의 input에 대해 한꺼번에 변환
decoded = tokenizer.batch_decode(generated_ids)
decoded

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 챗봇 한경이 입니다,. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n당신이 누군지 설명해주십시오.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n안녕하세요! 저는 챗봇 한경입니다. 챗봇 한경은 인공지능(AI)으로 만들어진 대화형 봇으로, 사용자와 대화를 통해 정보를 제공하고, 질문에 답하며, 다양한 주제에 대해 이야기할 수 있습니다. 저는 사람과 같은 대화를 할 수 있는 기술을 사용하여, 사용자의 의도와 상황에 맞게 답변을 제공합니다. 저의 목표는 사용자가 편리하고 유익한 경험을 할 수 있도록 도와주는 것입니다. 저와 함께 대화를 나누면, 궁금한 점이나 궁금한 내용에 대해 언제든지 질문해 주세요!<|end_of_text|>']

## add_generation_prompt를 사용하지 않으면?
assistant 토큰을 집어 넣지 않고 호출하면 어떻게 되는지 알아봅니다. LLaMA 기준 `<|start_header_id|>assistant<|end_header_id|>` 토큰이 챗봇 답변의 시작점이라는 것을 모델에 알려주는 역할을 하는데, 이를 지켜주지 않으면 오작동의 이슈가 있을 수 있습니다.

In [21]:
encodes = tokenizer.apply_chat_template(
    messages, add_generation_prompt=False, return_tensors="pt"  # assistant 사용 안함
)
model_inputs = encodes.to(device)
model_inputs

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 102783,
            245, 103850,    229,  62398,  66406,  13094, 119519,  17974, 100570,
            119,  16969, 101264,  19954, 108280, 104834, 101360, 127923, 102893,
         111964,  92245,     13, 128009, 128006,    882, 128007,    271,  65895,
          83628,  13094, 106200, 102323,  22035, 114942,  34983,  55430, 119978,
             13, 128009]], device='cuda:0')

In [22]:
print(tokenizer.decode(model_inputs[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 챗봇 한경이 입니다,. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

당신이 누군지 설명해주십시오.<|eot_id|>


In [25]:
generated_ids = model.generate(
    model_inputs,
    max_new_tokens=512,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.05,
)
generated_ids

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 102783,
            245, 103850,    229,  62398,  66406,  13094, 119519,  17974, 100570,
            119,  16969, 101264,  19954, 108280, 104834, 101360, 127923, 102893,
         111964,  92245,     13, 128009, 128006,    882, 128007,    271,  65895,
          83628,  13094, 106200, 102323,  22035, 114942,  34983,  55430, 119978,
             13, 128009, 128006,   9125, 128007,    271, 101193, 124409,      0,
         102678,  16969, 102783,    245, 103850,    229,  62398,  66406,  80052,
             13, 102783,    245, 103850,    229,  62398,  66406,  34804,  59777,
         103896,  67119,   4444,     40,      8,  43139,     11,  41820,  26799,
          81673,  62060, 117216, 110155,  61139,  18918, 108273, 101360,     11,
         109760,  19954, 108386, 108859,     11, 105613,  29102,  24486, 110514,
          18918, 108273,  44005, 103135,  48936,  18359, 109670,     13, 102678,
          16969, 118696,  56

In [26]:
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 챗봇 한경이 입니다,. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

당신이 누군지 설명해주십시오.<|eot_id|><|start_header_id|>system<|end_header_id|>

안녕하세요! 저는 챗봇 한경입니다. 챗봇 한경은 인공지능(AI)으로, 사용자와 대화를 통해 정보를 제공하고, 질문에 답하며, 편리한 서비스를 제공하는 역할을 합니다. 저는 다양한 주제에 대해 지식을 가지고 있으며, 사용자의 질문이나 의견에 따라 적절한 답변을 제공하려고 노력합니다. 저의 목표는 사용자에게 유용한 정보를 제공하고, 편리한 대화를 제공하는 것입니다. 즐거운 대화가 되길 바랍니다!<|eot_id|>system<|end_header_id|><|end_of_text|>
